# <font color="Blue">Abstract</font>

In this notebook, we delve into the utilization of LangChain and its 'prompt template' function.

**<font color="red">Our goal</font>** is to craft an effective prompt tailored to reviewing resumes in the context of specific jobs and companies.  

We used following knowledge:  
Prompt Engineering Skills:
1. Persona pattern (prompt 1)
2. Audience persona pattern (prompt 2)
3. Question refinement pattern (prompt 3, prompt 4, prompt 5)


# Install packages

In [ ]:
# Download required packages to Colab
!pip install -q langchain
!pip install -q openai
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q duckduckgo-search

# Import library

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain import ConversationChain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import os
from getpass import getpass

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# OpenAI API key

In [ ]:
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

··········


# Connect with Google Drive

# Create a .csv file to collect all template and results

In [ ]:
import csv

with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/prompt_result.csv', 'w', newline='') as csvfile:
    fieldnames = ['model', 'variable_combination', 'template', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    #writer.writerow({'input': user_input, 'output': result})

In [ ]:
with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv', 'w', newline='') as csvfile:
    fieldnames = ['comb_num','interviewer_role','company_name','resume','role_title','role_description','company_description']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

In [ ]:
import pandas as pd

In [ ]:
prompt_result = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/prompt_result.csv")

In [ ]:
var_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv")

In [ ]:
prompt_result

,model,variable_combination,template,output


In [ ]:
var_comb

,comb_num,interviewer_role,company_name,resume,role_title,role_description,company_description


# Prompt template for collect info

## Model selection: gpt-4

In [ ]:
chat = ChatOpenAI(model_name = "gpt-4") # temperature=0.9

NameError: ignored

In [ ]:
chat

## Variable define and track

### Prompt 1 (interviewer persona)

#### Set my_resume

In [ ]:
my_resume = """
EDUCATION
Vanderbilt University – Data Science Institute
Nashville, TN
Master of Science in Data Science
May 2024

Anticipated Coursework: Transformers in Theory and Practice, Data Science Algorithms, Launching the Venture
Relevant Coursework: EDA, Data Science Applications in Industry, Data Management Systems (SQL), Machine Learning
Cumulative GPA: 3.89/4.0
Saint Louis University
St. Louis, MO
Bachelor of Science in Data Science & Mathematics, Minor in Computer Science
May 2022

Relevant Coursework: Applied Regression, Applied Topology, Time Series, R for Spatial Analysis, Bayesian Statistics
Cumulative GPA: 3.9/4.0 (Dean’s List for 8 semesters)
EXPERIENCE
Washington University School of Medicine – Institute for Informatics
St. Louis, MO
Data Science Intern, Advisor: Dr. Ruijin Lu
May 2023 – Aug. 2023
North American Prodromal Synucleinopathy (NAPS) Project

Data Quality Control Automating: Implemented Python-based automation, reducing document download time by 99.2%, equivalent to saving a full week's work for one individual. Minimized manual & repetitive tasks, enhancing project efficiency by improving data quality, and aiding the group in identifying Alzheimer's Disease biomarkers across ten US sites.
Database & Data Management: Leveraged Python and Rave to optimize database updates, achieved a 75% time reduction updating birth records. Integrated with Amazon Textract API to halve the time of handwritten text extraction, enhanced overall efficiency, accuracy, and expanded system scalability. Minimized human bias through automated handwritten text recognition.
Collaboration & Problem Solving: Collaborated with data coordinators to proactively identify and address API integration and data accessibility issues, ensuring seamless data flow and enhanced system functionality. Contributed feedback to improve the quality of approximately 100 source documents, ensuring subsequent users access more refined and reliable data.
PROJECTS
Mock Interview Practice Application – Capstone Project
Nashville, TN
Project Designer & Project Manager & Code Contributor, Advisor: Dr. Jesse Spencer-Smith
Aug. 2023 – Present
Project Overview: Develop a Mock Interview Practice Application aim at motivating and helping current and future students at Data Science Institute prepare for job interviews by providing a platform where they can practice interview questions and receive feedback. The project involves designing the application architecture, managing the project timeline, and actively contributing to the codebase.
Roles and Responsibilities:

Project Designer: Conceptualize and design the application based on users’ needs and career coaches’ insights, ensuring a user-friendly interface and seamless navigation. Collaborated with team members to refine the project scope and objectives.
Project Manager: Oversaw the project timeline and coordinated team efforts to ensure timely completion. Implemented agile project management methodologies to facilitate effective collaboration and iterative development.
Code Contributor: Contributed to the development of the application by writing and reviewing code. Applied data science techniques to analyze user data and provide personalized feedback to users based on their performance.
Insurance Claims Prediction for Asurion – Data Science Teams Course
Nashville, TN
Data Science Candidate, Instructor: Dana Zhang
Jan. 2023 – May 2023
Project Overview: Developed a market applicable forecasting model using Python to predict iPhone 14 claim volume for each week of March 2023 for a phone insurance company. Worked in a four persons team, followed agile workflow and data science project flow.
Roles and Responsibilities:

Data Collection & Cleaning: Collected and cleaned weather, Apple shipment, stock and iPhone release date. Feature engineered seasonal promotion. Combined outside source data with Asurion’s phone claim data before March 2023.
Data Analysis & Modeling: Explored different visualizations for different phone brands, phone models, phone colors, and phone’s storage sizes to get insight about data. Developed basic regression models, time series models, and ensemble models.
Collaboration & Teamwork: Played both team leader role and team player role. Improved productivity of teamwork by updating, communicating and discussing constantly about each teammate’s progresses, challenges, findings, and team health.
Stakeholder Engagement & Presentation: Presented stakeholder results by transferring predicted numbers to estimated market value. Communicated with stakeholder the insights of advantages of overpredicting in the early stage of the device life cycle.
ADDITIONAL INFORMATION
Programming Skills: Python (Numpy, Pandas, Seaborn, Scikit-learn, PyTorch, Matplotlib), SQL (MySQLWorkbench, Database Design), NLP (Transformers, HuggingFace, LLMs, Prompt Engineering, LangChain), R (Statistical Analysis, Shiny)
Deliverable Skills: GitHub flow (Version Control, Project Board, Agile Project Management), Microsoft PowerPoint
Learning: Amazon Web Services (AWS), Google Cloud Platform (GCP), Tableau
"""

#### Specify job_description, company_info (DS, S&P)

In [ ]:
job_description = """
Job Description

This is a software development role to work with the stakeholders to develop AI models that will help the business to take smarter decisions. The role is for a software developer for AI/ML initiative. This role will support to an existing corporate finance IT team including cloud solutions, and digital platforms. This role also requires excellent decision-making, interpersonal, and communication skills.

Duties And Responsibilities
Implement machine learning algorithms
Collect and clean data
Evaluate the performance of AI/ML models
Work closely with the stakeholders to understand the business needs and develop solutions around these needs
Develop and deploy the AI models
Daily scrum update
Work closely with QA team to fix bugs
Basic understanding of automation
The Ideal Candidate Possesses
Bachelor’s degree from an accredited college or university, with major course work in computer science, information technology, or related field.
Master’s or advanced education is preferred.
3-5 years of experience in software development and testing
Experience with programming languages such as Python or Java is a plus.
Strong problem-solving skills.
Ability to work independently and as part of a team.
Basic understanding of machine learning
Passionate about learning emerging technologies
S&P Global states that the anticipated base salary range for this position is $60,000 - $90,000 USD. Base salary ranges may vary by geographic location.

This role is eligible to receive additional S&P Global benefits. For more information on the benefits we provide to our employees, visit https://spgbenefits.com/.

Flexible Working

We pride ourselves on our agility and diversity, and we welcome requests to work flexibly. For most roles, flexible hours and/or an element of remote working are usually possible. Please talk to us at interview about the type of arrangement that is best for you. We will always try to be adaptable wherever we can.

Return to Work

Have you taken time out for caring responsibilities and are now looking to return to work? As part of our Return to Work initiative (link to career site page when available), we are encouraging enthusiastic and talented returners to apply, and will actively support your return to the workplace.

Grade/Level (relevant for internal applicants only): 9

About Company Statement: S&P Global delivers essential intelligence that powers decision-making. We provide the world's leading organizations with the right data, connected technologies and expertise they need to move ahead. As part of our team, you'll help solve complex challenges that equip businesses, governments and individuals with the knowledge to adapt to a changing economic landscape.

EEO Statement

S&P Global is an equal opportunity employer committed to making all employment decisions without regard to race/ethnicity, gender, pregnancy, gender identity or expression, color, creed, religion, national origin, age, disability, marital status (including domestic partnerships and civil unions), sexual orientation, military veteran status, unemployment status, or any other basis prohibited by federal, state or local law. Only electronic job submissions will be considered for employment.

If you need an accommodation during the application process due to a disability, please send an email to: EEO.Compliance@spglobal.com and your request will be forwarded to the appropriate person.

Equal Opportunity Employer

S&P Global is an equal opportunity employer and all qualified candidates will receive consideration for employment without regard to race/ethnicity, color, religion, sex, sexual orientation, gender identity, national origin, age, disability, marital status, military veteran status, unemployment status, or any other status protected by law. Only electronic job submissions will be considered for employment.

If you need an accommodation during the application process due to a disability, please send an email to: EEO.Compliance@spglobal.com and your request will be forwarded to the appropriate person.

US Candidates Only:  The EEO is the Law Poster http://www.dol.gov/ofccp/regs/compliance/posters/pdf/eeopost.pdf describes discrimination protections under federal law.

IFTECH202.1 - Middle Professional Tier I (EEO Job Group)

Job ID: 289943

Posted On: 2023-09-18

Location: Virtual, New Jersey, United States
"""

In [ ]:
company_info = """
Overview
S&P Global (NYSE: SPGI) provides essential intelligence. We enable governments, businesses and individuals with the right data, expertise and connected technology so that they can make decisions with conviction. From helping our customers assess new investments to guiding them through sustainability and energy transition across supply chains, we unlock new opportunities, solve challenges and accelerate progress for the world.

We are widely sought after by many of the world’s leading organizations to provide credit ratings, benchmarks, analytics and workflow solutions in the global capital, commodity and automotive markets. With every one of our offerings, we help the world’s leading organizations plan for tomorrow, today. For more information, visit www.spglobal.com.

Our divisions include:
–	S&P Global Market Intelligence partners with customers to broaden their perspective and operate with confidence by bringing them leading data sources and technologies that embed insight in their daily work.

–	S&P Global Ratings offers critical insights for credit, risk and sustainable finance solutions that are essential to translating complexity into clarity, so market participants can uncover opportunities.

–	S&P Global Commodity Insights enables organizations to create long-term, sustainable value with data and insights for a complete view on the global energy and commodities markets.

–	S&P Global Mobility turns invaluable insights captured from automotive data to help our clients understand today’s market, reach more customers, and shape the future of automotive mobility.

–	S&P Dow Jones Indices provides iconic and innovative index solutions, bringing transparency to global capital markets.

–	S&P Global Engineering Solutions solves for tomorrow’s challenges today by transforming workflows and end-user experiences with data, insights and technology.

Website
http://www.spglobal.com
Industry
Financial Services
Company size
10,001+ employees
34,306 on LinkedIn
Includes members with current employer listed as S&P Global, including part-time roles. Also includes employees from subsidiary: Corporate Sustainability Assessment (CSA).
Headquarters
New York, NY
Specialties
Analytics, Credit ratings, Finance, Energy & commodities information, Intelligence, Commodities, Benchmarks, Portfolio and enterprise risk solutions, ESG, and Sustainable Finance
"""

#### Track variables in var_comb

In [ ]:
ex_interviewer_role = "recruiter" # hiring manager, data scientist, senior data scientist
ex_company_name = "S&P Global"
ex_resume = my_resume
ex_role_title = "Data Engineer - AI/ML (Virtual)"
ex_role_description = job_description
ex_company_description = company_info

ex_model = "gpt-4"

ex_var_comb = 1

In [ ]:
new_var_comb = pd.DataFrame({
    'comb_num': [ex_var_comb],
    'interviewer_role': [ex_interviewer_role],
    'company_name': [ex_company_name],
    'resume': [ex_resume],
    'role_title': [ex_role_title],
    'role_description': [ex_role_description],
    'company_description': [ex_company_description]
})

var_comb = var_comb.append(new_var_comb, ignore_index=True)

<ipython-input-26-2da0d4a65f09>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  var_comb = var_comb.append(new_var_comb, ignore_index=True)


In [ ]:
var_comb

,comb_num,interviewer_role,company_name,resume,role_title,role_description,company_description
0,1,recruiter,S&P Global,\nEDUCATION\nVanderbilt University – Data Scie...,Data Engineer - AI/ML (Virtual),\nJob Description\n\nThis is a software develo...,\nOverview\nS&P Global (NYSE: SPGI) provides e...


#### Track variables in prompt_result

In [ ]:
input_var_1 = ["interviewer_role", "company_name", "resume", "role_title", "role_description", "company_description"]

template_1 = """
You are a {interviewer_role}, for {company_name}. Please review the following resume {resume}.
For the role of {role_title}.
(optional: Here is the job description: {role_description})
(optional: Here is a description of the company: {company_description})
Give point by point feedback with rationale and suggested edits.
"""

In [ ]:
prompt = PromptTemplate(
    input_variables = input_var_1,
    template = template_1
)

In [ ]:
new_prompt = prompt.format(interviewer_role=ex_interviewer_role,
                           company_name=ex_company_name,
                           resume=ex_resume,
                           role_title=ex_role_title,
                           role_description=ex_role_description,
                           company_description=ex_company_description)
result = chat([HumanMessage(content=new_prompt)])

In [ ]:
print(result.content)

Overall, the resume is quite impressive. It shows a strong educational background, relevant coursework, and practical experience in data science. Here are some specific points of feedback:

1. Contact Information: Make sure to include your contact information, such as email and phone number, at the top of your resume. This is basic information that every employer will look for.

2. Education: Good use of space to list relevant coursework and highlight high GPA. Your degrees and coursework align well with the job requirements.

3. Experience: Great detail on specific achievements and responsibilities in previous roles. The use of quantifiable achievements is particularly effective. However, be sure to explain any industry-specific terms or acronyms (like NAPS or Rave) that might not be familiar to recruiters outside your immediate field.

4. Projects: The detailed breakdown of your roles and responsibilities in these projects demonstrates your ability to work on a team and manage comple

In [ ]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_1],
    'output': [result.content]
})

In [ ]:
prompt_result = prompt_result.append(new_prompt_result, ignore_index=True)

<ipython-input-33-924f668632ee>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  prompt_result = prompt_result.append(new_prompt_result, ignore_index=True)


In [ ]:
prompt_result

,model,variable_combination,template,output
0,gpt-4,1,"\nYou are a {interviewer_role}, for {company_n...","Overall, the resume is quite impressive. It sh..."


### Prompt 2 (interviewee persona)


#### var_comb (same as Prompt 1)

In [ ]:
#ex_interviewer_role = "recruiter" # hiring manager, data scientist, senior data scientist
#ex_company_name = "S&P Global"
#ex_resume = my_resume
#ex_role_title = "Data Engineer - AI/ML (Virtual)"
#ex_role_description = job_description
#ex_company_description = company_info

#ex_model = "gpt-4"

#ex_var_comb = 1

#### prompt_result

In [ ]:
template_2 = """
You are interviewing {company_name}, {role_title} role with a {interviewer_role}.
This is your resume {resume}. Please review the resume, job description, and company information below.
Give point by point feedback with rationale and suggested edits. Also, giving your examples based on your editing suggestions.
(optional: Here is the job description: {role_description})
(optional: Here is a description of the company: {company_description})
"""

In [ ]:
prompt = PromptTemplate(
    input_variables = input_var_1,
    template = template_2
)

In [ ]:
new_prompt = prompt.format(interviewer_role=ex_interviewer_role,
                           company_name=ex_company_name,
                           resume=ex_resume,
                           role_title=ex_role_title,
                           role_description=ex_role_description,
                           company_description=ex_company_description)
result = chat([HumanMessage(content=new_prompt)])

In [ ]:
print(result.content)

Feedback: 

1. Education: The educational background is highly relevant to the job role, however, the expected coursework can be removed as it is a future aspect and not directly related to the current job application. 

   Suggestion: Remove "Anticipated Coursework" and only include "Relevant Coursework". 

2. Experience: The experience section is detailed and shows relevant work in data science and AI/ML. However, it would be beneficial to highlight the use of Python or Java in the projects, as mentioned in the job description.

   Suggestion: Include specific instances of Python or Java usage in the projects. 

   Example: "Implemented Python-based automation...".

3. Projects: The projects showcase relevant experience and skills but could benefit from more specific details about the AI/ML models used, as well as any software development methodologies applied.

   Suggestion: Add details about specific AI/ML models and software development methodologies used in the projects.

   Exa

In [ ]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_2],
    'output': [result.content]
})

prompt_result = prompt_result.append(new_prompt_result, ignore_index=True)

<ipython-input-40-5bf6a7545659>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  prompt_result = prompt_result.append(new_prompt_result, ignore_index=True)


In [ ]:
prompt_result

,model,variable_combination,template,output
0,gpt-4,1,"\nYou are a {interviewer_role}, for {company_n...","Overall, the resume is quite impressive. It sh..."
1,gpt-4,1,"\nYou are interviewing {company_name}, {role_t...",Feedback: \n\n1. Education: The educational ba...


### [NOT GOOD] Prompt 3 (chatgpt refined prompt 2)

Here is my interation with Chat+:  
<font color='Green'>ME:</font>  
From now on, I will provide my prompt, you will suggest a better version of my prompt, and tell me your rationale.


<font color='Blue'>GPT:</font>   
Of course! Please provide your prompt, and I'll help you refine it.


<font color='Green'>ME:</font>  
prompt_2: You are interviewing {company_name}, {role_title} role with a {interviewer_role}. This is your resume {resume}. Please review the resume, job description, and company information below. Give point by point feedback with rationale and suggested edits. Also, giving your examples based on your editing suggestions. (optional: Here is the job description: {role_description}) (optional: Here is a description of the company: {company_description})


<font color='Blue'>GPT:</font>  
**Revised Prompt**:
"You are set to interview for the {role_title} position at {company_name} with an interviewer holding the title {interviewer_role}. Below you'll find your resume, the job description, and details about the company. Please review each section and provide detailed feedback, along with your rationale for each point. If possible, include examples to support your suggested edits. (Optional: Job description: {role_description}) (Optional: Company overview: {company_description})"
   
**Rationale**:
1. **Structural Changes**: I started the sentence with the main action ("You are set to interview") to make the reader's role immediately clear.
2. **Clarity**: I changed the ordering of some phrases to ensure each piece of information flows logically into the next.
3. **Consistency**: Used "job description" and "company overview" consistently to describe the optional inputs.
4. **Detailed Feedback**: I specified "detailed feedback" to emphasize the depth of feedback expected.
5. **Wording Choices**: Simplified some sentences for clarity and succinctness.

This revised prompt provides clearer directions and a smoother reading experience.

#### prompt_result

In [ ]:
template_3 = """
You are set to interview for the {role_title} position at {company_name} with an interviewer holding the title {interviewer_role}.
Below you'll find your resume, the job description, and details about the company.
Please review each section and provide detailed feedback, along with your rationale for each point.
If possible, include examples to support your suggested edits.
Your resume: {resume}.
Job description: {role_description}.
Company overview: {company_description}.
"""

In [ ]:
prompt = PromptTemplate(
    input_variables = input_var_1,
    template = template_3
)

new_prompt = prompt.format(interviewer_role=ex_interviewer_role,
                           company_name=ex_company_name,
                           resume=ex_resume,
                           role_title=ex_role_title,
                           role_description=ex_role_description,
                           company_description=ex_company_description)
result = chat([HumanMessage(content=new_prompt)])
print(result.content)

Resume Feedback:

1. Objective Statement: While your resume is impressive in terms of your academic qualifications and work experience, it lacks an objective statement. This is a valuable section where you can present your career aspirations and how your skills and experiences align with the company’s needs. For example, you could start with something like, "Motivated Data Science Master's student with proven experience in AI/ML, looking to apply my technical skills and problem-solving expertise in a challenging Data Engineering role at S&P Global."

2. Tailored Skills: Your resume is full of relevant skills, but it would be beneficial to tailor them more directly to the job description. For instance, the job description requires a basic understanding of machine learning and experience with Python, which you have, but they are buried in your resume. Make sure these skills stand out.

3. Software Development Experience: The job description emphasizes the need for experience in software 

In [ ]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_3],
    'output': [result.content]
})

prompt_result = prompt_result.append(new_prompt_result, ignore_index=True)

<ipython-input-51-37265705e151>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  prompt_result = prompt_result.append(new_prompt_result, ignore_index=True)


### [GOOD] Prompt 4 (prompt 3 with my revisement, interviewee persona)

#### prompt_result

In [ ]:
template_4 = """
You are set to interview for the {role_title} position at {company_name} with an interviewer holding the title {interviewer_role}.
Below you'll find your resume, the job description, and details about the company.
Please review each section of your resume and provide detailed feedback of how to tailor your resume based on the job description and company overview below, along with your rationale for each point.
If possible, include examples to support your suggested edits.
Your resume: {resume}.
Job description: {role_description}.
Company overview: {company_description}.
"""

In [ ]:
prompt = PromptTemplate(
    input_variables = input_var_1,
    template = template_4
)

new_prompt = prompt.format(interviewer_role=ex_interviewer_role,
                           company_name=ex_company_name,
                           resume=ex_resume,
                           role_title=ex_role_title,
                           role_description=ex_role_description,
                           company_description=ex_company_description)
result = chat([HumanMessage(content=new_prompt)])
print(result.content)

Resume feedback based on the job description and company overview:

1. Objective: Start your resume with a clear and concise objective that reflects your career goal and how it aligns with the responsibilities of the Data Engineer - AI/ML position at S&P Global. For example, "A highly skilled data science professional seeking to leverage my expertise in AI/ML for the Data Engineer position at S&P Global. Committed to developing innovative AI models to support business decision-making and drive operational efficiency."

2. Experience: Highlight your experience in software development, testing, and deploying AI models. For instance, under your internship at Washington University School of Medicine, you can rephrase "Database & Data Management" to "Developed, tested, and deployed AI models for managing and updating databases, reducing time spent on tasks by 75% and improving overall system efficiency."

3. Skills: You have listed Python as one of your programming skills. Given that the jo

In [ ]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_4],
    'output': [result.content]
})

prompt_result = prompt_result.append(new_prompt_result, ignore_index=True)

<ipython-input-46-e1628b0e6ddc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  prompt_result = prompt_result.append(new_prompt_result, ignore_index=True)


In [ ]:
prompt_result

,model,variable_combination,template,output
0,gpt-4,1,"\nYou are a {interviewer_role}, for {company_n...","Overall, the resume is quite impressive. It sh..."
1,gpt-4,1,"\nYou are interviewing {company_name}, {role_t...",Feedback: \n\n1. Education: The educational ba...
2,gpt-4,1,\nYou are set to interview for the {role_title...,Resume feedback based on the job description a...


### Prompt 5 (Question refinement patter of prompt 1, with my revisement, interviewer persona)

Here is my interation with Chat+:  
<font color='Green'>ME:</font>  
prompt_1: You are a {interviewer_role}, for {company_name}. Please review the following resume {resume}.
For the role of {role_title}.
(optional: Here is the job description: {role_description})
(optional: Here is a description of the company: {company_description})
Give point by point feedback with rationale and suggested edits

<font color='Blue'>GPT:</font>   
**Revised Prompt**:
"As an {interviewer_role} at {company_name}, please review the provided resume: {resume} for the {role_title} position.
(Optional: Job description: {role_description})
(Optional: Company overview: {company_description})
Provide itemized feedback, including the reasoning for each point and any recommended changes."

**Rationale**:
1. **Structural Changes**: I restructured the sentences to ensure that the main action ("review the provided resume") is clear and upfront.
2. **Consistency**: Standardized the optional input labels to "Job description" and "Company overview" for clarity and cohesion.
3. **Clarity**: Emphasized the role for which the resume should be reviewed by placing it close to the action.
4. **Succinctness**: The revised prompt is direct and concise while conveying the necessary information.

The refined prompt offers clear and structured guidance for the task at hand.


<font color='Green'>ME:</font>  
My revised prompt_1 after gpt's revisement:As an {interviewer_role} at {company_name}, please review the provided resume: {resume} for the {role_title} position.
Job description: {role_description}. Company overview: {company_description}.
Provide itemized feedback of how to tailor your resume based on the job description and company overview, including the reasoning for each point and any recommended changes.


<font color='Blue'>GPT:</font>  
**Revised Prompt**:
"As an {interviewer_role} with {company_name}, you're tasked with evaluating the given resume: {resume} for the {role_title} position. Below are the details:
- **Job Description**: {role_description}
- **Company Overview**: {company_description}

Please offer a detailed breakdown of suggestions to align the resume with the job description and company profile. Ensure each recommendation is accompanied by a rationale."

**Rationale**:
1. **Structural Changes**: I introduced bullet points for the job description and company overview to make the prompt easier to digest visually.
2. **Clarification**: Emphasized the main task ("evaluating the given resume") early in the prompt for immediate context.
3. **Succinctness and Flow**: Revised the wording for a more streamlined and coherent read.
4. **Emphasis**: Highlighted the main action words and key details to guide attention to the essential parts of the task.

The revised prompt offers clarity while maintaining the necessary details for the task.

#### prompt_result

In [ ]:
template_5 = """
As an {interviewer_role} with {company_name}, you're tasked with evaluating the given resume: {resume} for the {role_title} position.
Below are the details:
Job Description: {role_description}
Company Overview: {company_description}
Please offer a detailed breakdown of suggestions to align the resume with the job description and company profile.
Ensure each recommendation is accompanied by a rationale.
"""

In [ ]:
prompt = PromptTemplate(
    input_variables = input_var_1,
    template = template_5
)

new_prompt = prompt.format(interviewer_role=ex_interviewer_role,
                           company_name=ex_company_name,
                           resume=ex_resume,
                           role_title=ex_role_title,
                           role_description=ex_role_description,
                           company_description=ex_company_description)
result = chat([HumanMessage(content=new_prompt)])
print(result.content)

Recommendation 1: Highlight Relevant Experience
Rationale: The position requires 3-5 years of experience in software development and testing. The resume indicates experience in data science internships and projects but does not explicitly mention software development and testing experience. The candidate should highlight any relevant experience in software development and testing, even if it was part of a broader role or project. 

Recommendation 2: Emphasize Machine Learning Skills
Rationale: The job description mentions the implementation of machine learning algorithms and the evaluation of AI/ML models as key responsibilities. While the resume does mention machine learning in the education and experience sections, it would be helpful to elaborate on specific machine learning projects or experiences, detailing the models developed or evaluated.

Recommendation 3: Include Experience with Stakeholder Engagement
Rationale: The position requires working closely with stakeholders to under

In [ ]:
new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'variable_combination': [ex_var_comb],
    'template': [template_5],
    'output': [result.content]
})

prompt_result = prompt_result.append(new_prompt_result, ignore_index=True)

<ipython-input-50-05b5e7c17c3e>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  prompt_result = prompt_result.append(new_prompt_result, ignore_index=True)


# Save data
variable_combination.csv   
prompt_result.csv

In [ ]:
var_comb

,comb_num,interviewer_role,company_name,resume,role_title,role_description,company_description
0,1,recruiter,S&P Global,\nEDUCATION\nVanderbilt University – Data Scie...,Data Engineer - AI/ML (Virtual),\nJob Description\n\nThis is a software develo...,\nOverview\nS&P Global (NYSE: SPGI) provides e...


In [ ]:
prompt_result

,model,variable_combination,template,output
0,gpt-4,1,"\nYou are a {interviewer_role}, for {company_n...","Overall, the resume is quite impressive. It sh..."
1,gpt-4,1,"\nYou are interviewing {company_name}, {role_t...",Feedback: \n\n1. Education: The educational ba...
2,gpt-4,1,\nYou are set to interview for the {role_title...,Resume feedback based on the job description a...
3,gpt-4,1,\nAs an {interviewer_role} with {company_name}...,Recommendation 1: Highlight Relevant Experienc...
4,gpt-4,1,\nYou are set to interview for the {role_title...,Recommendation 1: Highlight Relevant Experienc...


In [ ]:
var_comb.to_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv", index=False)
prompt_result.to_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/prompt_result.csv", index=False)

# Appendix



## Code can be used later

In [ ]:
#prompt = (
#    SystemMessage(content="You are a nice pirate")
#    + HumanMessage(content="hi")
#    + AIMessage(content="what?")
#    + "{input}"
#)